In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:


### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data/pdf")



Found 1 PDF files to process

Processing: NPCK-2021-CATALOGUE-17.pdf
  ✓ Loaded 94 pages

Total documents loaded: 94


In [3]:


### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs



In [4]:
chunks=split_documents(all_pdf_documents)
#chunks

Split 94 documents into 143 chunks

Example chunk:
Content: 2
POTATO VARIETY CATALOGUE 2021...
Metadata: {'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Windows)', 'creationdate': '2021-10-19T23:58:06+03:00', 'moddate': '2021-10-20T00:02:46+03:00', 'trapped': '/False', 'source': '..\\data\\pdf\\NPCK-2021-CATALOGUE-17.pdf', 'total_pages': 94, 'page': 1, 'page_label': '2', 'source_file': 'NPCK-2021-CATALOGUE-17.pdf', 'file_type': 'pdf'}


In [2]:
### embedding And Vector Store DB
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

class EmbeddingManager:
    """Handles document embedding generation using SenteceTransformers"""

    def __init__(self,model_name: str = "all-MiniLM-L6-v2"):
        """Initialize the EmbeddingManager with a specified model. 
        Args:
            model_name Huggingface Model name for sentece Embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model."""
        try:
            print(f"Loading model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Emmbedding Dimension:, {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings for a list of texts.
        Args:
            texts List of texts to generate embeddings for
        Returns:
            np.ndarray Array of embeddings with shape (len(texts), embedding_dimension)
        """
        if not self.model:
            raise ValueError("Model not loaded. Call _load_model() first.")
        
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated {len(embeddings)} embeddings of shape {embeddings.shape}")
        return embeddings
    

    def get_sentence_embedding_dimension(self) -> int:
        """Get the dimension of the sentence embeddings."""
        if not self.model:
            raise ValueError("Model not loaded. Call _load_model() first.")
        return self.model.get_sentence_embedding_dimension()
    

embedding_manager = EmbeddingManager()
embedding_manager 


c:\Users\Stephen Mwangi\Projects\RAG Pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model: all-MiniLM-L6-v2
Model loaded successfully. Emmbedding Dimension:, 384



### VECTOR STORE


In [3]:
class VectorStore:
    """Manages document embedding in a ChromaDB vector store."""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """Initialize the VectorStore .
        Args:
            Collection name: Name of the vector database:
            persist_directory: Directory to persist the vector database
        """
        self.collection_name =collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize the ChromaDB client and collection."""
        try:
            print(f"Initializing ChromaDB client with persist directory: {self.persist_directory}")
            os.makedirs(self.persist_directory, exist_ok=True)

            self.client = chromadb.PersistentClient(path = self.persist_directory)

            ## Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata= {"description": "Collection of PDF document embeddings for RAG"}
                )
            print(f"Vector store collection '{self.collection_name}' initialized successfully.")
            print(f"Existing collections: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing ChromaDB: {e}")
            raise
    
    def add_documents(self, documents: List[Any],embeddings:np.ndarray) -> List[str]:
        """Add documents to the ChromaDB collection after generating embeddings.

        Args:
            documents List of Document objects to add
            embeddings: Corresponding embeddings for the documents
            
        """
        if len(documents) != len (embeddings):
            raise ValueError("Number of documents and embeddings must match.")
        
        ids = []
        metadatas = []
        documents_text = []
        embeddings_lists = []

        for i, (doc,embedding) in enumerate(zip(documents,embeddings)):
            #Generate a unique ID for each document
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            ## Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length']  = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_lists.append(embedding.tolist())
            
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_lists,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store.")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding document {doc_id} to collection: {e}")
            raise
            
            
vector_store = VectorStore()

Initializing ChromaDB client with persist directory: ../data/vector_store
Vector store collection 'pdf_documents' initialized successfully.
Existing collections: 143


In [8]:
### Convert the text to embeddings and add to vector store
texts = [doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings = embedding_manager.generate_embeddings(texts)

## Add to vector store
vector_store.add_documents(chunks,embeddings)

Batches: 100%|██████████| 5/5 [00:01<00:00,  2.73it/s]

Generated 143 embeddings of shape (143, 384)
Successfully added 143 documents to vector store.
Total documents in collection: 143


## Retriever Pipeline From VectorStore

In [4]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """Initialize the RAGRetriever with a vector store and embedding manager.
        Args:
            vector_store Instance of the VectorStore class
            embedding_manager Instance of the EmbeddingManager class for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
        
        
    def retrieve(self, query: str, top_k: int = 5,score_threshhold:float = 0.0) -> List[Dict[str, Any]]:
            """Retrieve top_k relevant documents for a given query.
            Args:
                query The user query string
                top_k Number of top relevant documents to retrieve
            Returns:
                List of dictionaries containing retrieved documents with metadata
            """
            print(f"Retrieving top {top_k} documents for query: '{query}'")
            print(f"Score Threshold: {score_threshhold}")
            
            # Generate embedding for the query
            query_embedding = self.embedding_manager.generate_embeddings([query])[0]
            
            # Perform similarity search in the vector store
            try:
                results = self.vector_store.collection.query(
                    query_embeddings=[query_embedding.tolist()],
                    n_results=top_k,
                    include=['metadatas', 'documents', 'distances']
                )
                
                #Process results
                retrieved_docs = []
                if results['documents'] and results['documents'][0]:
                    documents = results['documents'][0]
                    metadatas = results['metadatas'][0]
                    distances = results['distances'][0]
                    ids = results['ids'][0]
                    
                    for i, (doc_id,document, metadata, distance) in enumerate(zip(ids,documents, metadatas, distances)):
                        #similarity_score = 1 - distance  # Convert distance to similarity score
                        if (1 - distance) >= score_threshhold:  # Convert distance to similarity score
                            retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': 1 - distance,  # Convert distance to similarity score
                            'distance': distance,
                            'rank': i + 1})
                    print(f"Retrieved {len(retrieved_docs)} documents (after applying score threshold)")  
                else:
                    print("No documents retrieved.")
                
                return retrieved_docs     
                    
            except Exception as e:
                print(f"Error during retrieval: {e}")
                return [] 


rag_retriever = RAGRetriever(vector_store, embedding_manager)  


In [5]:
rag_retriever.retrieve("What is the National Potato Council of Kenya (NPCK)?",top_k=3,score_threshhold=0.1)

Retrieving top 3 documents for query: 'What is the National Potato Council of Kenya (NPCK)?'
Score Threshold: 0.1


Batches: 100%|██████████| 1/1 [00:00<00:00, 10.80it/s]

Generated 1 embeddings of shape (1, 384)
Retrieved 3 documents (after applying score threshold)


[{'id': 'doc_2af49bfc_18',
  'content': '9\nPOTATO VARIETY CATALOGUE 2021\nNATIONAL POTATO COUNCIL OF KENYA\nThe National Potato Council of Kenya (NPCK) is a Public Private Partnership (PPP) and \na multi-stakeholder organization whose responsibility is to help plan, organize, and \nco-ordinate potato value chain activities and develop the subsector into a robust, \ncompetitive, and self-regulating industry. The Potato Council organizational structure \nenables it to draw synergies from a wide membership, representing all stakeholders \nand actors in the potato industry.\nNPCK VISION STATEMENT\nPotato industry is a leading contributor to increased incomes, food security, and \nimproved welfare in Kenya\nNPCK MISSION STATEMENT\nIt is NPCK’s mission to help coordinate and regulate Kenya’s potato industry, and help \nimprove the industry’s profitability and the livelihoods of its various stakeholders.\nNPCK OBJECTIVES\nThe NPCK Strategic Plan is anchored on six objectives:',
  'metadata':

## Integration Vectordb Context pipeline with LLM Output

In [6]:
### Simple RAG pipeline with Groq LLM

from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.environ["GROQ_API_KEY"]  

### initialize the Grow LLM (set your GROQ_API_KEY in .env file)
llm = ChatGroq(api_key=groq_api_key,model="gemma2-9b-it",temperature=0.1,max_tokens=1024)

## 2. Define a simple RAG function to combine retrieval and generation

def rag_simple(query,rag_retriever,llm,top_k=3):
    """Simple RAG function to retrieve documents and generate an answer using LLM"""
    # Step 1: Retrieve relevant documents
    retrieved_docs = rag_retriever.retrieve(query, top_k=top_k)
    
    # Step 2: Combine retrieved documents into a context string
    context = "\n\n".join([doc['content'] for doc in retrieved_docs]) if retrieved_docs else ""
    
    if not retrieved_docs:
        return "No relevant documents found."
    
    
    # Step 3: Create a prompt for the LLM
    prompt = f"Use the following context to answer the question concisely:\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
    
    #print(f"\nGenerated Prompt for LLM:\n{prompt[:500]}...\n")  # Print first 500 characters of the prompt
    
    # Step 4: Generate an answer using the LLM
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    return response.content

In [7]:
answer = rag_simple("What is the National Potato Council of Kenya (NPCK)?",rag_retriever,llm,top_k=3)
print(f"\nGenerated Answer:\n{answer}\n")

Retrieving top 3 documents for query: 'What is the National Potato Council of Kenya (NPCK)?'
Score Threshold: 0.0


Batches: 100%|██████████| 1/1 [00:00<00:00, 153.03it/s]

Generated 1 embeddings of shape (1, 384)
Retrieved 3 documents (after applying score threshold)



Generated Answer:
The National Potato Council of Kenya (NPCK) is a Public Private Partnership (PPP) that coordinates and regulates Kenya's potato industry to improve profitability and the livelihoods of its stakeholders.  




### Enhanced RAG Pipeline

In [8]:
def rag_advanced(query,rag_retriever,llm,top_k=3,min_score=0.2,return_context=False):
    """Advanced RAG function to retrieve documents and generate an answer using LLM with source attribution, confidence score, and optional full context"""
    
    # Step 1: Retrieve relevant documents
    retrieved_docs = rag_retriever.retrieve(query, top_k=top_k,score_threshhold=min_score)
    if not retrieved_docs:
        return {'answer':"No relevant documents found.",'sources':[], 'confidence':0.0, 'context':""}
    
    context = "\n\n".join([doc['content'] for doc in retrieved_docs])
    # Step 2: Combine retrieved documents into a context string with source attribution
    
    sources = [{
        'source': doc['metadata'].get('source_file',doc['metadata'].get('source','unknown source')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:120] + "..." # First 100 characters as preview
    } for doc in retrieved_docs]
    
    confidence = max(doc['similarity_score'] for doc in retrieved_docs) 
    
        
    # Step 3: Create a prompt for the LLM
    prompt = f"Use the following context to answer the question concisely and cite sources:\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer (with sources):"
    
    #print(f"\nGenerated Prompt for LLM:\n{prompt[:500]}...\n")  # Print first 500 characters of the prompt
    
    # Step 4: Generate an answer using the LLM
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {'answer': response.content, 'sources': sources, 'confidence': confidence}
    if return_context:
        output['context'] = context
    
    return output

In [9]:
result = rag_advanced("What is the National Potato Council of Kenya (NPCK)?",rag_retriever,llm,top_k=3,min_score=0.1,return_context=True)

print("Answer:",result['answer'])
print("Sources:"), result['sources']
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving top 3 documents for query: 'What is the National Potato Council of Kenya (NPCK)?'
Score Threshold: 0.1


Batches: 100%|██████████| 1/1 [00:00<00:00, 158.08it/s]

Generated 1 embeddings of shape (1, 384)
Retrieved 3 documents (after applying score threshold)


Answer: The National Potato Council of Kenya (NPCK) is a Public Private Partnership (PPP) and multi-stakeholder organization responsible for planning, organizing, and coordinating potato value chain activities in Kenya. Its goal is to develop a robust, competitive, and self-regulating potato industry.  (Source: NPCK Potato Variety Catalogue 2021, Foreword) 



Sources:
Confidence: 0.5058606266975403
Context Preview: 9
POTATO VARIETY CATALOGUE 2021
NATIONAL POTATO COUNCIL OF KENYA
The National Potato Council of Kenya (NPCK) is a Public Private Partnership (PPP) and 
a multi-stakeholder organization whose responsibility is to help plan, organize, and 
co-ordinate potato value chain activities and develop the subs
